In [1]:
import cv2
import numpy as np

In [5]:
def histogram_matching(source, target):
    # Convert the images from BGR to LAB color space
    source_lab = cv2.cvtColor(source, cv2.COLOR_BGR2LAB)
    target_lab = cv2.cvtColor(target, cv2.COLOR_BGR2LAB)

    matched = np.zeros_like(target_lab)

    for i in range(3):
        # Compute the histogram and cumulative distribution function (CDF)
        src_hist, src_bins = np.histogram(source_lab[:, :, i].flatten(), 256, [0, 256])
        tgt_hist, tgt_bins = np.histogram(target_lab[:, :, i].flatten(), 256, [0, 256])

        src_cdf = src_hist.cumsum()
        src_cdf_normalized = src_cdf * (255 / src_cdf[-1])

        tgt_cdf = tgt_hist.cumsum()
        tgt_cdf_normalized = tgt_cdf * (255 / tgt_cdf[-1])

        # Create a lookup table
        lut = np.zeros(256)
        g_j = 0
        for g_i in range(256):
            while g_j < 255 and tgt_cdf_normalized[g_j] < src_cdf_normalized[g_i]:
                g_j += 1
            lut[g_i] = g_j

        # Apply the lookup table to the target image
        matched[:, :, i] = cv2.LUT(target_lab[:, :, i], lut.astype(np.uint8))

    # Convert the LAB image back to BGR color space
    result = cv2.cvtColor(matched, cv2.COLOR_LAB2BGR)

    return result

In [10]:
# Load the source and target images
source = cv2.imread('Images/source.jpg')
target = cv2.imread('Images/target.jpg')

In [11]:
# Perform histogram matching
result = histogram_matching(source, target)

# Save the result
cv2.imwrite('histogram_matched.jpg', result)

True